# Imported Libraries

In [23]:
import camelot
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium import webdriver

## 1) Extracting Sparkassen Info from PDF

Used the following function to fix the issue of having all data in one cell

In [224]:
# https://stackoverflow.com/questions/45846765/efficient-way-to-unnest-explode-multiple-list-columns-in-a-pandas-dataframe

def explode(df, lst_cols, fill_value=''):
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]

In [244]:
tables = camelot.read_pdf('Sparkassenrangliste_2018.pdf',pages='all')

# Creating new df to append to
df = pd.DataFrame()

# Iterate over each table in the 10 pages
for table in tables:
    table = table.df
    table.columns = ['Rang','Institut','Sitz','Verband','Bilanzsumme gemäß Bilanzstatistik','Kundeneinlagen','Spareinlagen', 'Kundenkredite','Anzahl Mitarbeiter','Sparkassenstellen (einschließlich SB)']
    table.drop(index=[0,1,2],inplace=True)
    table = table.apply(lambda text: text.str.split('\n'))
    table.reset_index(inplace=True)
    table.drop(columns='index',inplace=True)
    table = explode(table,list(table.columns))
    df = df.append(table)

df.set_index('Rang',inplace=True)

# Get rid of '.' to convert to numeric
df[list(df.columns)[3:]] = df[list(df.columns)[3:]].apply(lambda number: number.str.replace('.',''))

# Convert colukns into integers
for column in list(df.columns)[3:]:
    df[column] = pd.to_numeric(df[column])

# Remove any potential whitespace to have clean text
for column in list(df.columns)[:3]:
    df[column] = df[column].str.strip()
    
# Save df as csv for later use in the scarper
df.to_csv('sparkassen_rangliste_cleaned.csv')

# 2) Getting for each Sparkassen Name the website URL with Selenium 

In [ ]:
df = pd.read_csv('sparkassen_rangliste_cleaned.csv')

sparkaseen_links = []

# Setting driver to Safari
driver = webdriver.Chrome(
    executable_path='/Users/felixvemmer/OneDrive/Dokumente/Hobbies/Programming/Python/chromedriver')

# Extracting sparkassen from df and constructing search url
sparkassen = df['Institut'].to_list()
google_search_url = 'https://www.google.de/search?q='

for sparkasse in sparkassen:
    google_url = google_search_url+sparkasse.replace(' ','+')
    driver.get(google_url)
    driver.find_element_by_css_selector('h3').click() 
    target_url = driver.current_url
    sparkaseen_links.append(target_url)
    
df['links'] = sparkaseen_links

# Correcting one record manually
df.at['122','links'] = 'https://www.sparkasse-hegau-bodensee.de/de/home.html'

# Save df as csv for later use in the scarper
df.to_csv('sparkassen_rangliste_cleaned.csv')

## 3) Use Scrapy to test and generate different patterns for target url

## 4) Join verified urls to existing dataframe for final scraping

In [122]:
sparkassen = pd.read_csv('sparkassen_rangliste_cleaned.csv')
verified_urls = pd.read_csv('/Users/felixvemmer/Desktop/sparkassen_accounts/verified_links.csv')

In [123]:
sparkassen['domain'] = sparkassen['links'].str.split('/',expand=True)[2].str.split('www.',expand=True)[1]
verified_urls['domain'] = verified_urls['verified_url'].str.split('/', expand=True)[2].str.split('www.', expand=True)[1]

In [138]:
final.iloc[[89,14,44,169]]

,Rang,Institut,Sitz,Verband,Bilanzsumme gemäß Bilanzstatistik,Kundeneinlagen,Spareinlagen,Kundenkredite,Anzahl Mitarbeiter,Sparkassenstellen (einschließlich SB),links,domain,verified_url
89,87,Sparkasse Bamberg,Bamberg,SVB,4103245,3326524,1000166,2605242,815,48,https://www.sparkasse-bamberg.de/en/home.html,sparkasse-bamberg.de,NaN
14,15,Sparkasse Nürnberg,Nürnberg,SVB,11469244,9220404,2144547,6750665,1914,107,https://www.sparkasse-nuernberg.de/en/home.html,sparkasse-nuernberg.de,NaN
44,43,Sparkasse Ulm,Ulm,SVBW,6346182,4830784,1817440,4060486,1053,80,https://www.sparkasse-ulm.de/en/home.html,sparkasse-ulm.de,NaN
169,165,Sparkasse Bad Tölz-Wolfratshausen,Bad Tölz,SVB,2484363,2013194,400194,1795153,452,29,https://www.spktw.de/en/home.html,spktw.de,NaN


In [129]:
final = pd.merge(sparkassen, verified_urls, how='left').drop_duplicates()

In [130]:
final[final['verified_url'].isnull()]['links']

14     https://www.sparkasse-nuernberg.de/en/home.html
44           https://www.sparkasse-ulm.de/en/home.html
89       https://www.sparkasse-bamberg.de/en/home.html
169                  https://www.spktw.de/en/home.html
Name: links, dtype: object

In [ ]:
sample_not_working_urls = [
   'https://www.sparkasse-nuernberg.de/de/home/privatkunden/girokonto.html?n=true&stref=hnav',
    'https://www.sparkasse-ulm.de/de/home/privatkunden/girokonto.html?n=true&stref=hnav',
]

In [99]:
final_set = set()
for link in sample_not_working_urls:
        target_part = link.split('/home')[1].split('.html?')[0]
        final_set.add(target_part + '.html?')